


#### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [40]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup


from selenium import webdriver


from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException


import requests


import re

In [41]:

driver = webdriver.Chrome('chromedriver.exe')

In [27]:
# Opening the URL
driver.get('https://www.amazon.in/')
# Asking the user to input the keywords user wants to search
search = input('Enter the product you want to search : ')

Enter the product you want to search : guitar


In [28]:
 # Locating search bar
search_bar = driver.find_element_by_id("twotabsearchtextbox")
# clearing search bar
search_bar.clear()
# sending user input to search bar
search_bar.send_keys(search)       
# Locating search_button by xpath
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       
search_button.click()                                                                

#### Q2. 2.	In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [29]:
#start_page = 0
#end_page = 3
urls = []
for page in range(0,4):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')    
            if url[0:4]=='http':                
                urls.append(url)               
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      
        if nxt_button.text == 'Next→':                                            
            nxt_button.click()                                                    
            time.sleep(5)                                                         
          
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  
            break
            
    except StaleElementReferenceException as e:             
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        
        driver.get(next_page)                               

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [30]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [31]:
for url in urls[:4]:
    driver.get(url)                                                        
    
    # scraping data for Brand 
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
        
      # scraping data for Name of the product
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')     
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
      # scraping data for rating
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    # scraping no. of Ratings 
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    # scraping Price 
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
        
        # scraping Return/Exchange 
    try:                                                                                     
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
        
         # scraping Expected Delivery 
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')        
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    # scraping Availability 
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    # scraping Other Details 
    try:                                                                                    
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    #scraping the url
    prod_dict['URL'].append(url)                                                            
    time.sleep(2)

In [34]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Brand: Intern,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",3.9 out of 5 stars,"6,225 ratings","₹2,290.00",7 Days Replacement,"Monday, July 26",In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.9 out of 5 stars,467 ratings,"₹2,299.00",7 Days Replacement,"Tuesday, July 27",In stock.,Material: Wood || Colour: Blue || Acoustic...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5 stars,925 ratings,-,7 Days Replacement,"Tuesday, July 27",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the Kadence Store,Kadence Acoustica Series Semi Acoustic Ash Woo...,3.9 out of 5 stars,601 ratings,-,7 Days Replacement,"Tuesday, July 27",In stock.,Product Type : Acoustic Guitar Fretboard mater...,https://www.amazon.in/gp/slredirect/picassoRed...


In [37]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(search))
driver.close()

#### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [48]:
data_list=['fruits','cars','Machine Learning']
path='chromedriver.exe'
for data in data_list:
    driver=webdriver.Chrome(path)
    driver.get('https://www.google.com/imghp?hl=en')
    search_bar = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
    search_bar.send_keys(data)
    search_btn = driver.find_element_by_xpath('//span[@class="z1asCe MZy1Rb"]')
    search_btn.click()

    for i in range(2000):

        driver.execute_script("window.scrollBy(0,10000)")

    images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

    img_urls = []
    for image in images:
        source = image.get_attribute('src')
        if source is not None:
            if(source[0:4] == 'http'):
                img_urls.append(source)
    import requests
    for i in range(len(img_urls)):
        if i>100:
            break
        print("Downloading {0} of {1} images".format(i,100))
        response = requests.get(img_urls[i])
        
        file = open(r"E:\img\img_data"+str(i)+".jpg","wb")
        file.write(response.content)   

driver.close()

#### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [50]:
driver = webdriver.Chrome('chromedriver.exe')

In [51]:

prod = input(" Enter the name of the mobile phone you want to search : ")
driver.get('https://www.flipkart.com/')
time.sleep(3)
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    
search_bar.send_keys(prod)     
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')   
search_button.click()        

 Enter the name of the mobile phone you want to search : pixel 4A


In [52]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [53]:
len(flip_urls)

24

In [54]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [55]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
    
                                                               

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=dc40cc4b-e727-45c9-8b59-1edf80e31f5f.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=tou322kce80000001627102169850&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/redmi-9-power-mighty-black-64-gb/p/itm481ab234a6553?pid=MOBFYZ94UWPWQRNF&lid=LSTMOBFYZ94UWPWQRNFD8SF5T&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=dc40cc4b-e727-45c9-8b59-1edf80e31f5f.MOBFYZ94UWPWQRNF.SEARCH&ppt=hp&ppn=homepage&ssid=tou322kce80000001627102169850&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/redmi-9-power-fiery-red-64-gb/p/itmcd78b9bc04aa9?pid=MOBFYZF8XVSHPK5M&lid=LSTMOBFYZF8XVSHPK5MMUYMJE&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=s

In [56]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

24 24 24 24 24


In [57]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df
driver.close()

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Battery Type,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,-,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,9 Power,Mighty Black,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹10,999",https://www.flipkart.com/redmi-9-power-mighty-...
2,REDMI,9 Power,Fiery Red,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹10,999",https://www.flipkart.com/redmi-9-power-fiery-r...
3,REDMI,9 Power,Electric Green,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹10,999",https://www.flipkart.com/redmi-9-power-electri...
4,SAMSUNG,GALAXY M31S,Mirage Black,8 GB,128 GB,Primary Camera,-,16.51 cm (6.5 inch),1080 x 2400$$pixel,-,Octa Core,-,-,"₹20,579",https://www.flipkart.com/samsung-galaxy-m31s-m...
5,REDMI,9 Power,Blazing Blue,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹13,499",https://www.flipkart.com/redmi-9-power-blazing...
6,REDMI,9 Power,Electric Green,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹13,499",https://www.flipkart.com/redmi-9-power-electri...
7,SAMSUNG,M31,Ocean Blue,6 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,lithium-ion,"₹17,538",https://www.flipkart.com/samsung-m31-ocean-blu...
8,SAMSUNG,Galaxy M31,Space Black,8 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,lithium-ion,"₹18,198",https://www.flipkart.com/samsung-galaxy-m31-sp...
9,REDMI,9 Power,Mighty Black,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,Lithium-polymer,"₹13,499",https://www.flipkart.com/redmi-9-power-mighty-...


#### Q5. 5.	Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [58]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                        
search = driver.find_element_by_id("searchboxinput")                       
search.clear()                                                             
time.sleep(2)
search.send_keys(city)                                                     
button = driver.find_element_by_id("searchbox-searchbutton")               
button.click()                                                      
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))
driver.close()

Enter City Name : Pune
URL Extracted:  https://www.google.co.in/maps/place/Pune,+Maharashtra/@18.4637699,73.7623226,15z/data=!4m5!3m4!1s0x3bc2bf2e67461101:0x828d43bf9d9ee343!8m2!3d18.5204303!4d73.8567437
Latitude = 18.4637699, Longitude = 73.7623226


#### Q6.	Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [65]:
driver = webdriver.Chrome('chromedriver.exe')

In [66]:
driver.get('https://trak.in/')

In [67]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [68]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [69]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [70]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")
driver.close()

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(4)

url = 'https://www.digit.in/'
driver.get(url)
time.sleep(6)

In [ ]:
topten = driver.find_element_by_xpath('//div[@class="search"]').click()

In [ ]:

topten = driver.find_element_by_xpath('//input[@id="globalPageSearchText"]')

In [ ]:

topten.send_keys("LAPTOP")

In [ ]:
Button=driver.find_element_by_xpath('//input[@placeholder="Search Here..."]')

In [ ]:
Button.click()

In [ ]:

#url=[]
#url=driver.find_element_by_xpath('//div[@class="searchPage"]/a').get_attribute('href')

In [ ]:
url=[]

for i in driver.find_elements_by_xpath('//div[@class="searchPage"]/a'):
  url.append(i.get_attribute('href'))
url

In [ ]:
OS=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
        A=driver.find_element_by_xpath('//div[@class="value"]')
        OS.append(A.text)
    
    except NoSuchElementException :
        OS.append('-')

In [ ]:

Dis=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
      B=driver.find_element_by_xpath('//div[@class="Specs-Wrap  notspecs "]/ul/li[2]')
      Dis.append(B.text)
    
    except NoSuchElementException :
      Dis.append('-')

In [ ]:

Processor=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
      C=driver.find_element_by_xpath('//div[@class="Specs-Wrap  notspecs "]/ul/li[3]')
      Processor.append(C.text)
    
    except NoSuchElementException :
      Processor.append('-')

In [ ]:

Memory1=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
      D=driver.find_element_by_xpath('//div[@class="Specs-Wrap  notspecs "]/ul/li[4]')
      Memory1.append(D.text)
    
    except NoSuchElementException :
      Memory1.append('-')

In [ ]:
price1=[]


for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
      E=driver.find_element_by_xpath('//div[@class="Block-price"]')
      price1.append(E.text)
    
    except NoSuchElementException :
      price1.append('-')

In [ ]:
Laptop=pd.DataFrame({})
Laptop['OS']=OS[:10]
Laptop['Display']=Dis[:10]
Laptop['Processor']=Processor[:10]
Laptop['Menory']=Memory1[:10]
Laptop['Price']=price[:10]

Laptop

In [ ]:
driver.close()

#### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 9

In [ ]:
path='chromedriver.exe'
driver=webdriver.Chrome(path)
driver.get('https://www.forbes.com/billionaires/')

In [ ]:
forbes_rank=[]
forbes_name=[]
forbes_net_worth=[]
forbes_age=[]
forbes_citizenship=[]
forbes_source=[]
forbes_industry=[]

In [ ]:
#scraping data for rank
try:
    rank=driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        forbes_rank.append(i.text)
except:
    forbes_rank.append('--')

In [ ]:
#scraping data for name
try:
    name=driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        forbes_name.append(i.text)
except:
    forbes_name.append('--')

In [ ]:
#scraping data for net worth
try:
    net_worth=driver.find_elements_by_xpath("//div[@class='netWorth']")
    for i in net_worth:
        forbes_net_worth.append(i.text)
except:
    forbes_net_worth.append('--')

In [ ]:
#scraping data for age
try:
    age=driver.find_elements_by_xpath("//div[@class='age']")
    for i in age:
        forbes_age.append(i.text)
except:
    forbes_age.append('--')

In [ ]:
#scraping data for country
try:
    citizenship=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citizenship:
        forbes_citizenship.append(i.text)
except:
    forbes_citizenship.append('--')

In [ ]:
#scraping data for source of income
try:
    source=driver.find_elements_by_xpath("//div[@class='source-column']")
    for i in source:
        forbes_source.append(i.text)
except:
    forbes_source.append('--')

In [ ]:
#scraping data for industry
try:
    industry=driver.find_elements_by_xpath("//div[@class='category']")
    for i in industry:
        forbes_industry.append(i.text)
except:
    forbes_industry.append('--')

In [ ]:
forbes_info=pd.DataFrame({})
forbes_info['Rank']=forbes_rank
forbes_info['Name']=forbes_name
forbes_info['Net Worth']=forbes_net_worth
forbes_info['age']=forbes_age
forbes_info['Citizenship']=forbes_citizenship
forbes_info['Source']=forbes_source
forbes_info['Industry']=forbes_industry
forbes_info

In [ ]:
forbes_info.to_csv("Forbes information.csv")
driver.close()

#### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [87]:
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/')
driver.implicitly_wait(10)

In [88]:
search_box=driver.find_element_by_xpath('//input[@ id="search"]')

In [ ]:
search_box.send_keys("https://youtu.be/3u7MQz1EyPY")

In [91]:
search_botton=driver.find_element_by_xpath('//button[@id="search-icon-legacy"]').click()

In [92]:
search_video=driver.find_element_by_xpath('//yt-formatted-string[@class="style-scope ytd-video-renderer"]').click()

44

In [93]:
comments=[]

for i in driver.find_elements_by_xpath('//yt-formatted-string[@class="style-scope ytd-comment-renderer"]'):

    comments.append(i.text.replace('\n',''))

In [94]:
youtubeBI=pd.DataFrame({})



youtubeBI['Comments']=comments[:500]

['IN\nSKIP NAVIGATION\nSIGN IN\n0:01 / 0:15\nHow To Open a Door\n2.3M views\n92K\n6.9K\nSHARE\nSAVE\nHowToBasic\n16.6M subscribers\nSUBSCRIBE\nWhat Basic Thing Would You Like To Learn?\nSuggest In The Comments Below!\n\nClick Here To Eggscribe! --►\nSHOW MORE\n3:25\nHow To Fix a Water Damaged Laptop\nHowToBasic\n36M views\n11 months ago\n3:05:15\nHow To Make Pulled Pork\nHowToBasic\n3M views\n1 year ago\n6:10\nI am a Rider | Imran khan Satisfya | Now You See Me 2 Scene | Gaddi Lamborghini | I am a Rider song\nJack Jaxon\n220M views\n1 year ago\n2:12\nHow To Solve a Rubik\'s Cube in 10 Seconds\nHowToBasic\n5.9M views\n6 months ago\n10:03\nHow To Make a Vegan Curry\nHowToBasic\n17M views\n3 years ago\n8:58\nUnbreakable Ice Cream Safe- How to make cool stuff (I made a class!)\nMark Rober\n13M views\n7 months ago\n1:25\nHow To Flip an Egg\nHowToBasic\n32M views\n6 years ago\n8:09\nHow To Make a Curried Egg Sandwich\nHowToBasic\n4.5M views\n11 months ago\n2:41\nHow To Fix a Printer\nHowToBa

In [ ]:
Timeof1update=[]
try:
      for i in  driver.find_elements_by_xpath('//div[@class="style-scope ytd-comment-renderer"]'):
        if 'ago'in  i.text.strip():
             Timeof1update.append(i.text.replace('\n',''))
    
except NoSuchElementException :
     Timeof1update.append('-')

In [ ]:

Timeof1update

In [ ]:
youtubeBI['Timeof1update']=Timeof1update[:500]
youtubeBI
driver.close()

#### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.  

In [ ]:

driver = webdriver.Chrome("chromedriver.exe")
time.sleep(4)

url = 'https://www.hostelworld.com/'
driver.get(url)
time.sleep(6)

In [ ]:
search_Bar=driver.find_element_by_xpath('//input[@id="search-input-field"]')

search_Bar.send_keys('London, England')

In [ ]:
search_Button=driver.find_element_by_xpath('//button[@ id="search-button"]')
search_Button.click()

In [ ]:
hostel_name=[]

for i in driver.find_elements_by_xpath('//h2[@ class="title title-6"]/a'):

    hostel_name.append(i.text.replace('\n',''))


In [ ]:
distance=[]

for i in driver.find_elements_by_xpath('//span[@ class="description"]'):

    distance.append(i.text.replace('\n',''))
    


In [ ]:
ratings=[]

for i in driver.find_elements_by_xpath('//div[@ class="score orange big"]'):

    ratings.append(i.text.replace('\n',''))
    


In [ ]:
total_reviews=[]

for i in driver.find_elements_by_xpath('//div[@ class="reviews"]'):
    total_reviews.append(i.text.replace('\n',''))
    


In [ ]:
overall_reviews=[]

for i in driver.find_elements_by_xpath('//div[@ class="keyword"]/span'):

    overall_reviews.append(i.text.replace('\n',''))
    


In [ ]:
privates_price=[]

for i in driver.find_elements_by_xpath('//div[@ class="price-col"]'):
     if  'Privates'   in i.text.strip():
        privates_price.append(i.text.replace('\n',''))
    


In [ ]:
dorms_price=[]

for i in driver.find_elements_by_xpath('//div[@ class="price-col"]'):
     if  'Dorms'   in i.text.strip():
        dorms_price.append(i.text.replace('\n',''))
    


In [ ]:
facilities=[]

for i in driver.find_elements_by_xpath('//span[@ class="label"]'):
     if  'Free'   in i.text.strip():
       facilities.append(i.text.replace('\n',''))
    


In [ ]:
url=[]

for i in driver.find_elements_by_xpath('//h2[@class="title title-6"]/a'):
  url.append(i.get_attribute('href'))


In [ ]:
property_description=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
        E=driver.find_element_by_xpath('//div[@style="-webkit-line-clamp:12;"]')
        property_description.append(E.text)
        
    
    except NoSuchElementException :
        property_description.append('-'

In [ ]:
property_short_description=[]
for i in url[:10]:
    driver.get(i)
    time.sleep(3)

    try:
        E=driver.find_element_by_xpath('//div[@class="content collapse-content"]')
        property_short_description.append(E.text)
        
    
    except NoSuchElementException :
        property_short_description.append('-')

In [ ]:
import pandas as pd
Londan=pd.DataFrame({})
Londan['hostel_name']=hostel_name[:10]
Londan['distance']=distance[:10]
Londan['ratings']=ratings[:10]
Londan['total_reviews']=total_reviews[:10]
Londan['overall_reviews']=overall_reviews[:10]
Londan['privates_price']=privates_price[:10]
Londan['dorms_price']=dorms_price[:10]
Londan['facilities']=facilities[:10]
Londan['property_short_description']=property_short_description[:10]
Londan

driver.close()